In [63]:
import torch
from torch.utils.data import IterableDataset, DataLoader, Subset
from datetime import datetime as dt, timedelta
import pandas as pd
import os
import random
import numpy as np
import torch.nn as nn
from pandas import DataFrame as df
import mplfinance as mpf

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

seed = 42  # choose any seed you prefer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

cuda


In [71]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = f'../csvfiles/{item}'
        self.item = item
        self.timespan = timespan
        start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [1, 4]  # Selecting open and close prices
        self.filenames = self.get_filenames()

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def get_filenames(self):
        filenames = []
        for date in self.dates:
            filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
            if os.path.exists(filename):
                filenames.append(filename)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        return torch.tensor(df.values, dtype=torch.float32)  # Return open and close prices


def sliding_window_pct(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 400 + 1):  # Create windows of 100 rows each
            window = tensor[i:i+400]
            # Calculate the percentage change for each pair in the window
            pct_change = (window[:, 1] - window[:, 0]) * 100 / window[:, 0]
            windows.append(pct_change)
    # Only keep the first 100 windows if there are more than 100
    return torch.stack(windows)

def sliding_window_binary(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 400 + 1):  # Create windows of 100 rows each
            window = tensor[i:i+400]
            binary_change = (window[:, 1] > window[:, 0]).float()  # Calculate the binary change
            windows.append(binary_change)
    # Only keep the first 100 windows if there are more than 100
    return torch.stack(windows)


In [65]:
# Create the dataset
train_dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')
train_loader_pct = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)
train_loader_binary = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)

test_dataset = PriceDataset('ETHUSDT', '1m', '2021-03-01', '2023-04-30')
test_loader_pct = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)
test_loader_binary = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)

In [66]:
# Size of the Dataset
print(f'Train dataset size: {len(train_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

# Size of the DataLoader (i.e., number of batches)
print(f'Train dataloader size: {len(train_loader_binary)}')
print(f'Test dataloader size: {len(test_loader_binary)}')

# Size of the tensor output by the Dataset
sample_tensor = train_dataset[0]
print(f'Shape of the tensor output by train_dataset: {sample_tensor.shape}')

# Size of the tensor output by the DataLoader
for batch in train_loader_binary:
    print(f'Shape of the tensor output by train_loader: {batch.shape}')
    break  # we break after the first batch

first_batch = next(iter(train_loader_binary))

first_batch = next(iter(train_loader_pct))
print(first_batch[0])


Train dataset size: 791
Test dataset size: 791
Train dataloader size: 791
Test dataloader size: 791
Shape of the tensor output by train_dataset: torch.Size([1440, 2])
Shape of the tensor output by train_loader: torch.Size([941, 500])
tensor([ 2.8056e-01,  2.2834e-01, -5.0143e-01, -2.0211e-01, -9.1939e-03,
        -1.4603e-01,  3.4472e-02,  1.3322e-01,  4.6631e-01,  3.0887e-02,
        -1.6091e-01,  1.4937e-01,  3.2405e-02, -4.9789e-02,  4.7234e-01,
         2.6495e-01,  5.9878e-01,  6.2401e-01, -8.1065e-03,  1.9066e-02,
         7.1394e-02, -1.7840e-01,  4.7845e-02, -1.7791e-01,  2.7748e-01,
        -5.2244e-02,  8.0417e-03,  2.0064e-02, -3.8876e-02,  3.8866e-02,
         4.9639e-01, -3.5896e-02,  9.8013e-02,  2.6715e-01, -1.0436e-01,
        -1.1240e-01, -7.0540e-02, -9.4212e-02,  4.2307e-02,  1.2704e-01,
        -3.0736e-02, -4.9750e-02,  1.0694e-02, -1.5194e-01, -1.7663e-01,
        -1.4057e-02, -1.3757e-01,  5.6558e-02, -7.6856e-02,  5.1975e-02,
        -8.3747e-04,  6.6676e-02,  2

In [67]:
class PriceChangePrediction(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=100, output_dim=1, num_layers=4):
        super(PriceChangePrediction, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout = 0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


In [68]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch in train_loader:
        batch = batch.unsqueeze(-1).to(device)  # Adds an extra dimension
        inputs = batch[:, :-1, :]
        targets = batch[:, 1:, :]  # The targets are the next time-steps
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.unsqueeze(-1).to(device)  # Adds an extra dimension
            inputs = batch[:, :-1, :]
            targets = batch[:, 1:, :]  # The targets are the next time-steps
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)  # Return average loss



def train_and_evaluate(model, model_name, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        train(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} \t | \t {model_name} \t | \t Validation Loss: {val_loss:.10f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{model_name}.pth')
            best_val_loss = val_loss



In [74]:
# Create the models
percentage_model = PriceChangePrediction().to(device)
binary_model = PriceChangePrediction().to(device)

# Define the loss functions
percentage_criterion = nn.MSELoss()
binary_criterion = nn.BCEWithLogitsLoss()

# Define the optimizers
percentage_optimizer = torch.optim.Adam(percentage_model.parameters(), lr=0.01)
binary_optimizer = torch.optim.Adam(binary_model.parameters(), lr=0.01)

try: 
    # Load the saved models and optimizers
    percentage_checkpoint = torch.load('models/percentage_model.pth')
    binary_checkpoint = torch.load('models/binary_model.pth')

    percentage_model.load_state_dict(percentage_checkpoint['model_state_dict'])
    binary_model.load_state_dict(binary_checkpoint['model_state_dict'])

    percentage_optimizer.load_state_dict(percentage_checkpoint['optimizer_state_dict'])
    binary_optimizer.load_state_dict(binary_checkpoint['optimizer_state_dict'])
except:
    pass

# Set the number of epochs
epochs = 10

# Train and evaluate PriceChangePrediction model
train_and_evaluate(percentage_model, 'percentage_model', train_loader_pct, test_loader_pct, percentage_criterion, percentage_optimizer, epochs, device=device)

# Train and evaluate PriceDirectionPrediction model
train_and_evaluate(binary_model, 'binary_model', train_loader_binary, test_loader_binary, binary_criterion, binary_optimizer, epochs, device=device)

# Get the output of binary_model for a random input
random_input = torch.randn(1, 100, 1).to(device)  # Create a random tensor of shape (1, 100, 1)
random_output = binary_model(random_input)  # Get the output

print(random_output)


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.04 GiB (GPU 0; 8.00 GiB total capacity; 3.91 GiB already allocated; 1.20 GiB free; 4.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Epoch 1/1 | percentage_model Validation Loss: 0.0161349802
Epoch 1/1 | binary_model Validation Loss: 0.6933783825

In [69]:
random_input = torch.randn(1, 100, 1).to(device)  # Create a random tensor of shape (1, 100, 1)
random_output = model1(random_input)  # Get the output

print(random_output)

tensor([[[0.0007],
         [0.0030],
         [0.0033],
         [0.0031],
         [0.0030],
         [0.0029],
         [0.0029],
         [0.0029],
         [0.0029],
         [0.0029],
         [0.0030],
         [0.0030],
         [0.0030],
         [0.0030],
         [0.0030],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.0031],
         [0.

In [73]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()